# OLA: Explanatory Data Analysis with SQL

In [56]:
# Configuration
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus
load_dotenv()
encoded_password = quote_plus(os.getenv('password'))
db_uri = (
    f"postgresql://{os.getenv('user')}:"
    f"{encoded_password}@"
    f"{os.getenv('host')}:"
    f"{os.getenv('port')}/"
    f"{os.getenv('dbname')}"
)
%reload_ext sql
%sql $db_uri
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%config SqlMagic.displaycon = False

## Check Data Intigrity

### Check if the row count matches with the dataset provided

In [41]:
%%sql
SELECT COUNT(*) FROM rides;

1 rows affected.


count
103024


This is the exact no. of rows that is present in the initial dataset. Hence, no data loss happend during import.

### Check if there is any duplicate rows

In [42]:
%%sql
SELECT COUNT(DISTINCT booking_id)
FROM public.rides;

1 rows affected.


count
103024


All the booking ids are distinct hence no duplicate rows are there. Let's dive into the data and try to answer some EDA questions.

## SQL Questions
Now as the data is clean, let's try to answer some questions through the power of MagicSQL.

1. Retrieve all successful bookings:

To do this, first list all the booking messages.

In [43]:
%%sql
SELECT DISTINCT booking_status
FROM public.rides;

4 rows affected.


booking_status
Success
Canceled by Customer
Canceled by Driver
Driver Not Found


So, we need to look for all the booking with status "Success". Instead of list all the booking details, just check percentage of successfull bookings.

In [44]:
%%sql
SELECT
    ROUND((
        COUNT(*) FILTER (WHERE booking_status = 'Success') * 100.0 / COUNT(*)
    ), 2) AS successful_booking_percentage
FROM public.rides;

1 rows affected.


successful_booking_percentage
62.09


So, 62.09% of the bookings were successful.

2. Find the average ride distance for each vehicle type:

In [45]:
%%sql
SELECT vehicle_type, ROUND(AVG(ride_distance_km), 2) AS average_distance_km
FROM public.rides
GROUP BY vehicle_type
ORDER BY average_distance_km DESC;

7 rows affected.


vehicle_type,average_distance_km
Prime Sedan,15.76
eBike,15.58
Bike,15.53
Mini,15.51
Prime Plus,15.45
Prime SUV,15.27
Auto,6.24


The "Auto" has the lowest run among all types of rides with the others sharing almost equal millage.

3. Get the total number of cancelled rides by customers:

In [46]:
%%sql
SELECT COUNT(*)
FROM public.rides
WHERE
    booking_status = 'Canceled by Customer';

1 rows affected.


count
10499


4. List the top 5 customers who booked the highest number of rides:

In [47]:
%%sql
SELECT
    customer_id,
    COUNT(*) AS ride_count
FROM
    public.rides
GROUP BY customer_id
ORDER BY ride_count DESC
LIMIT 5;

5 rows affected.


customer_id,ride_count
CID954071,5
CID836942,4
CID782883,4
CID356460,4
CID288207,4


5. Get the number of rides cancelled by drivers due to personal and car-related issues:

To get this, let's list all the cancellation messages first.

In [48]:
%%sql
SELECT DISTINCT driver_cancellation_reason
FROM public.rides
WHERE booking_status = 'Canceled by Driver';

4 rows affected.


driver_cancellation_reason
More than permitted people in there
Personal & Car related issue
Customer related issue
Customer was coughing/sick


Now, get the number of rides cancelled by the driver for "Personal & Car related issue".

In [49]:
%%sql
SELECT
    COUNT(*)
FROM
    public.rides
WHERE
    driver_cancellation_reason = 'Personal & Car related issue';

1 rows affected.


count
6542


That's 6542 number of cases where the driver cancelled the ride for personal and car related issues.

6. Find the maximum and minimum driver ratings for Prime Sedan bookings:

In [50]:
%%sql
SELECT
    MAX(driver_rating) AS maximum_rating,
    MIN(driver_rating) AS minimum_rating
FROM
    public.rides
WHERE
    vehicle_type = 'Prime Sedan';

1 rows affected.


maximum_rating,minimum_rating
5,3


7. Retrieve all rides where payment was made using UPI:

First list all the payment methods used.

In [51]:
%%sql
SELECT
    DISTINCT payment_method
FROM public.rides;

5 rows affected.


payment_method
None
Debit Card
UPI
Credit Card
Cash


Here also, instead of listing all the booking information, let's find out the percentage of each payment methods.

In [52]:
%%sql
SELECT
    payment_method,
    ROUND((
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER()
    ), 2) AS booking_percentage
FROM public.rides
GROUP BY payment_method
ORDER BY booking_percentage DESC;

5 rows affected.


payment_method,booking_percentage
None,37.91
Cash,33.99
UPI,25.12
Credit Card,2.36
Debit Card,0.61


Though Debit Card transactions are least, payment through cash is still on the higher side.

8. Find the average customer rating per vehicle type:

In [53]:
%%sql
SELECT
    vehicle_type,
    ROUND(AVG(customer_rating), 2) as avg_customer_rating
FROM
    public.rides
GROUP BY vehicle_type
ORDER BY avg_customer_rating DESC
;

7 rows affected.


vehicle_type,avg_customer_rating
Prime Plus,4.01
Auto,4.00
Prime Sedan,4.00
Mini,4.00
Prime SUV,4.00
eBike,3.99
Bike,3.99


9. Calculate the total booking value of rides completed successfully:

Here, istead of just showing the booking value for only successfull rides, let's list total booking values for all booking statuses which will give us a comprehensive comparison's

In [54]:
%%sql
SELECT
    booking_status,
    COUNT(*) AS total_bookings,
    SUM(booking_value)::money as total_booking_value
FROM
    public.rides
GROUP BY booking_status
ORDER BY total_booking_value DESC;

4 rows affected.


booking_status,total_bookings,total_booking_value
Success,63967,"$35,080,467.00"
Canceled by Driver,18434,"$10,183,427.00"
Canceled by Customer,10499,"$5,770,901.00"
Driver Not Found,10124,"$5,499,819.00"


10. List all incomplete rides along with the reason

Here also instead of listing all the booking details and cluttering the notebook, let's check the percentage of each reason for unsuccessfull bookings.

In [55]:
%%sql
SELECT
    incomplete_ride_reason,
    ROUND(COUNT(*) * 100 / SUM(COUNT(*)) OVER (), 2) AS percentage_incomplete_bookings
FROM
    public.rides
WHERE
    is_incomplete_ride = TRUE
GROUP BY incomplete_ride_reason
ORDER BY percentage_incomplete_bookings DESC;

3 rows affected.


incomplete_ride_reason,percentage_incomplete_bookings
Customer Demand,40.78
Vehicle Breakdown,40.52
Other Issue,18.70
